# Creation of Insights 

This file contains all the code required to generate bsuiness insights. The code below relies on a clean transaction dataset being loaded. The dataset is cleaned in ''.

In [1]:
#Libraries
#Dataframe and Array
import pandas as pd
import numpy as np

#Graphing/ Charting/ Visualisations
import matplotlib as plt

In [2]:
df = pd.csv_read('../data/****.csv')

In [ ]:
#Revenue by Day of Week - All Countries
df_day_of_week_rev = df_days_week.groupby(['Weekday'])['Rev'].agg('sum')
df_day_of_week_rev.sort_values(ascending=False)#.head(10)

In [9]:
df.InvoiceDate.min()

Timestamp('2010-12-01 08:26:00')

In [10]:
df.InvoiceDate.max()

Timestamp('2011-12-09 12:50:00')

In [14]:
#Top 10 Countries
df_top_countries_stg = df.groupby(['Country'])['Rev'].agg('sum')
df_top_countries = df_top_countries_stg.sort_values(ascending=False).head(10).to_frame()
df_top_countries

,Rev
Country,
United Kingdom,8.187806e+06
Netherlands,2.846615e+05
EIRE,2.632768e+05
Germany,2.216982e+05
France,1.974039e+05
Australia,1.370773e+05
Switzerland,5.638535e+04
Spain,5.477458e+04
Belgium,4.091096e+04


In [15]:
df_top_countries = df_top_countries.reset_index()


In [16]:
top_country_rev = []
for i in df_top_countries['Country'][:3]:
    top_country_rev.append(i)
top_country_rev    

['United Kingdom', 'Netherlands', 'EIRE']

In [17]:
#Top 10 Description by Revenue - All Countries
df_top_items_rev = df.groupby(['Description'])['Rev'].agg('sum')
df_top_items_rev.sort_values(ascending=False).head(10)

Description
DOTCOM POSTAGE                        206245.48
REGENCY CAKESTAND 3 TIER              164762.19
WHITE HANGING HEART T-LIGHT HOLDER     99668.47
PARTY BUNTING                          98302.98
JUMBO BAG RED RETROSPOT                92356.03
RABBIT NIGHT LIGHT                     66756.59
POSTAGE                                66230.64
PAPER CHAIN KIT 50'S CHRISTMAS         63791.94
ASSORTED COLOUR BIRD ORNAMENT          58959.73
CHILLI LIGHTS                          53768.06
Name: Rev, dtype: float64

In [18]:
#Top 10 Description by Quantity - All Countries
df_top_items_vol = df.groupby(['Description'])['Quantity'].agg('sum')
df_top_items_vol.sort_values(ascending=False).head(10)

Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     53847
JUMBO BAG RED RETROSPOT               47363
ASSORTED COLOUR BIRD ORNAMENT         36381
POPCORN HOLDER                        36334
PACK OF 72 RETROSPOT CAKE CASES       36039
WHITE HANGING HEART T-LIGHT HOLDER    35317
RABBIT NIGHT LIGHT                    30680
MINI PAINT SET VINTAGE                26437
PACK OF 12 LONDON TISSUES             26315
PACK OF 60 PINK PAISLEY CAKE CASES    24753
Name: Quantity, dtype: int64

In [19]:



for i in top_country_rev:
    print(df.loc[df['Country'] == i].groupby(['Description'])['Quantity'].agg('sum').sort_values(ascending=False).head(3))
    

Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS    48326
JUMBO BAG RED RETROSPOT              43167
POPCORN HOLDER                       34365
Name: Quantity, dtype: int64
Description
RABBIT NIGHT LIGHT      4801
SPACEBOY LUNCH BOX      4528
DOLLY GIRL LUNCH BOX    4132
Name: Quantity, dtype: int64
Description
PACK OF 72 RETROSPOT CAKE CASES    1728
60 TEATIME FAIRY CAKE CASES        1536
VINTAGE SNAP CARDS                 1492
Name: Quantity, dtype: int64


In [24]:
#Revenue by Day of Week - All Countries
df_day_of_week_rev = df_days_week.groupby(['Weekday'])['Rev'].agg('sum')
df_day_of_week_rev.sort_values(ascending=False)#.head(10)

Weekday
Thursday     2112519.000
Tuesday      1966182.791
Wednesday    1734147.010
Monday       1588609.431
Friday       1540610.811
Sunday        805678.891
Name: Rev, dtype: float64

In [26]:
#Customers with Negative Qty
#Identify Customer/ Product mix that are negative - i.e. potential returns from outside reporting period

df_cust_neg_qty = df.groupby(['CustomerID', 'StockCode'])['Quantity'].agg('sum').to_frame()
df_cust_neg_qty.reset_index(inplace=True)
df_cust_neg_qty = df_cust_neg_qty.loc[(df_cust_neg_qty['Quantity'] <= 0)]
df_cust_neg_qty

df_new = df[~(df['StockCode'].isin(df_cust_neg_qty.StockCode) & df['CustomerID'].isin(df_cust_neg_qty.CustomerID))]

df_new.shape

(434567, 10)

In [29]:
#Stratisfying Customers based on Spend During Reporting Period

# PART 1
rev_by_cust = df_new.groupby("CustomerID")["Rev"].sum().to_frame()
rev_by_cust

rev_by_cust['quantile'] = pd.qcut(rev_by_cust['Rev'], 100, labels=False)
rev_by_cust

# Emanuele's way didnt work. 
conditions = [
    (rev_by_cust['quantile'] >= 75 ) & (rev_by_cust['quantile'] <= 95),
    (rev_by_cust['quantile'] > 95)]
choices = ['Preferred', 'VIP']
rev_by_cust['Status'] = np.select(conditions, choices, default='Regular')
print(rev_by_cust)


                Rev  quantile     Status
CustomerID                              
12347.0     4310.00        95  Preferred
12348.0     1437.24        79  Preferred
12349.0     1457.55        79  Preferred
12350.0      294.40        28    Regular
12352.0      245.13        24    Regular
12353.0       89.00         4    Regular
12354.0     1079.40        72    Regular
12355.0      459.40        45    Regular
12356.0     2487.43        89  Preferred
12357.0     6207.67        97        VIP
12358.0      928.06        67    Regular
12359.0     1446.13        79  Preferred
12360.0     2302.06        88  Preferred
12361.0      174.90        15    Regular
12362.0     1679.43        82  Preferred
12363.0      552.00        50    Regular
12364.0     1208.10        75  Preferred
12365.0      140.34        11    Regular
12367.0      150.90        12    Regular
12370.0     3425.69        93  Preferred
12371.0     1527.96        80  Preferred
12372.0     1196.04        74    Regular
12373.0      324

In [30]:
#Top Returning Customers
df_top_returners = df.groupby(['CustomerID'])['Quantity'].agg('sum').to_frame()
df_top_returners.reset_index(inplace=True)
df_top_returners = df_top_returners.loc[(df_top_returners['Quantity'] < 0)].sort_values(by='Quantity', ascending=True)
df_top_returners

,CustomerID,Quantity
3103,16546.0,-303
2578,15823.0,-283
1384,14213.0,-244
3245,16742.0,-189
2892,16252.0,-158
3650,17307.0,-144
3829,17548.0,-132
4350,18256.0,-70
264,12666.0,-56
2436,15638.0,-52


In [32]:
#Top Invoice Values by Rev
df_top_invoices_rev = df.groupby(['InvoiceNo', 'CustomerID'])['Rev'].agg('sum').to_frame()
df_top_invoices_rev.reset_index(inplace=True)
df_top_invoices_rev = df_top_invoices_rev.sort_values(by='Rev', ascending=False)
df_top_invoices_rev

,InvoiceNo,CustomerID,Rev
18503,581483,16446.0,168469.60
1909,541431,12346.0,77183.60
7926,556444,15098.0,38970.00
12419,567423,17450.0,31698.16
8112,556917,12415.0,22775.93
14459,572209,18102.0,22206.00
12411,567381,17450.0,22104.80
10843,563614,12415.0,21880.44
5463,550461,15749.0,21535.90
14371,572035,14646.0,20277.92


In [33]:
#Top Invoice Values by Qty
df_top_invoices_qty = df.groupby(['InvoiceNo', 'CustomerID'])['Quantity'].agg('sum').to_frame()
df_top_invoices_qty.reset_index(inplace=True)
df_top_invoices_qty = df_top_invoices_qty.sort_values(by='Quantity', ascending=False)
df_top_invoices_qty

,InvoiceNo,CustomerID,Quantity
18503,581483,16446.0,80995
1909,541431,12346.0,74215
8112,556917,12415.0,15049
10612,563076,14646.0,14730
14371,572035,14646.0,13392
12419,567423,17450.0,12572
17360,578841,13256.0,12540
6437,552883,14646.0,12266
10843,563614,12415.0,12196
10344,562439,12931.0,11848


In [34]:
#Top Customer by Rev
df_top_customer_rev = df.groupby(['CustomerID'])['Rev'].agg('sum').to_frame()
df_top_customer_rev.reset_index(inplace=True)
df_top_customer_rev = df_top_customer_rev.sort_values(by='Rev', ascending=False)
df_top_customer_rev['CustomerID'] = df_top_customer_rev['CustomerID'].astype(int)
df_top_customer_rev

,CustomerID,Rev
1703,14646,279489.02
4233,18102,256438.49
3758,17450,187482.17
1895,14911,132572.62
55,12415,123725.45
1345,14156,113384.14
3801,17511,88125.38
3202,16684,65892.08
1005,13694,62653.10
2192,15311,59419.34


In [36]:
#Find descriptions with negative works like 'damaged' 'fees' 'mouldy', 'faulty', 'wet', 'lost', 'missing','wrong', 'broke'

In [45]:
#df_new['CustomerID'].isna().sum()

In [46]:
#df_new.dropna(subset=['CustomerID'],inplace=True)


In [50]:
df_new.to_csv('../data/cleaned_df2.csv')